# Set Up

In [373]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

pd.options.display.max_colwidth = 150

# Data Retrieval

In [87]:
url = "https://en.wikipedia.org/wiki/2023_Alberta_general_election#Opinion_polling"

df1 = pd.read_html(url)[23]
df2 = pd.read_html(url)[24]
df3 = pd.read_html(url)[25]


In [88]:
df1.columns = ['Pollster', 'Client', 'Dates', 'Source', 'UCP', 'NDP', 'Alberta', 'Liberal', 'IPA', 'Green', 'WIP', 'Others', 'Margin of error', 'Sample size', 'Polling method', 'Lead']
df1.head(3)

,Pollster,Client,Dates,Source,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Margin of error,Sample size,Polling method,Lead
0,EKOS,NaN,"May 19–26, 2023",[p 1][p 2],50.4%,47.3%,0.8%,0.4%,—,—,—,1.1%,2.35%,1741,Online/IVR,3.1%
1,Leger,Postmedia,"May 23–25, 2023",[p 3],49%,46%,—,—,—,—,—,—,3.1%,1011,Online,3%
2,Mainstreet Research,NaN,"May 22–25, 2023",[p 4],48%,46%,2%,—,—,2%,1%,1%,2.3%,1734,IVR,2%


In [89]:
df2.columns = ['Pollster', 'Client', 'Dates', 'Source', 'UCP', 'NDP', 'Alberta', 'Liberal', 'IPA', 'Green', 'WIP', 'Others', 'Margin of error', 'Sample size', 'Polling method', 'Lead']
df2.head(3)

,Pollster,Client,Dates,Source,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Margin of error,Sample size,Polling method,Lead
0,Ipsos,Global News,"April 26–30, 2023",[p 36],48%,44%,4%,—,—,—,—,4%,3.2%,1200,Telephone / Online,4%
1,ThinkHQ,NaN,"April 25–29, 2023",[p 37],46%,46%,5%,1%,—,—,—,2%,2.5%,1529,Online,Tie
2,Abacus Data,NaN,"April 21–25, 2023",[p 38],46%,46%,5%,—,—,—,—,4%,3.1%,1000,Online,Tie


In [90]:
df3.insert(8, 'IPA', 0)
df3.columns = ['Pollster', 'Client', 'Dates', 'Source', 'UCP', 'NDP', 'Alberta', 'Liberal', 'IPA', 'Green', 'WIP', 'Others', 'Margin of error', 'Sample size', 'Polling method', 'Lead']
df3.head(3)

,Pollster,Client,Dates,Source,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Margin of error,Sample size,Polling method,Lead
0,ThinkHQ,NaN,"May 19–23, 2023",[p 112],43%,49%,6%,2%,0,–,–,1%,3%,973,Online,6%
1,Mainstreet Research,NaN,"April 17–18, 2023",[p 113],45.6%,44.1%,–,–,0,–,–,–,2.4%,"<1,651",Smart IVR,1.5%
2,Janet Brown Opinion Research,CBC,"Mar 23–April 6, 2023",[p 114],42%,47%,3%,1%,0,1%,1%,–,3.1%,1000,Telephone / Online,5%


In [265]:
polls = pd.concat([df1, df2, df3], ignore_index = True)
polls = polls.drop(columns=['Client', 'Source', 'Margin of error', 'Sample size'])
polls.head()

,Pollster,Dates,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Polling method,Lead
0,EKOS,"May 19–26, 2023",50.4%,47.3%,0.8%,0.4%,—,—,—,1.1%,Online/IVR,3.1%
1,Leger,"May 23–25, 2023",49%,46%,—,—,—,—,—,—,Online,3%
2,Mainstreet Research,"May 22–25, 2023",48%,46%,2%,—,—,2%,1%,1%,IVR,2%
3,Mainstreet Research,"May 21–24, 2023",49%,45%,2%,—,—,2%,1%,1%,IVR,4%
4,Janet Brown Opinion Research,"May 12–24, 2023",52%,44%,1%,1%,1%,1%,<1%,<1%,Telephone/Online,8%


# Data Cleaning

In [266]:
polls = polls.replace('—', '0%')
polls = polls.replace('<1%', '0%')
polls = polls.dropna()
polls = polls[~polls.Pollster.str.contains("Pollster")]
polls.head()

,Pollster,Dates,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Polling method,Lead
0,EKOS,"May 19–26, 2023",50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,Online/IVR,3.1%
1,Leger,"May 23–25, 2023",49%,46%,0%,0%,0%,0%,0%,0%,Online,3%
2,Mainstreet Research,"May 22–25, 2023",48%,46%,2%,0%,0%,2%,1%,1%,IVR,2%
3,Mainstreet Research,"May 21–24, 2023",49%,45%,2%,0%,0%,2%,1%,1%,IVR,4%
4,Janet Brown Opinion Research,"May 12–24, 2023",52%,44%,1%,1%,1%,1%,0%,0%,Telephone/Online,8%


In [267]:
polls[['Start Date', 'End Date Temp']] = polls['Dates'].str.split('–', 1, expand=True)
polls[['End Date', 'Year']] = polls['End Date Temp'].str.split(', ', 1, expand=True)
polls = polls.drop(columns=['End Date Temp'])
polls.insert(13, 'Start Month', polls["Start Date"].str.split(" ", 0, expand = True)[0])
polls.head()

,Pollster,Dates,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Polling method,Lead,Start Date,Start Month,End Date,Year
0,EKOS,"May 19–26, 2023",50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,Online/IVR,3.1%,May 19,May,26,2023
1,Leger,"May 23–25, 2023",49%,46%,0%,0%,0%,0%,0%,0%,Online,3%,May 23,May,25,2023
2,Mainstreet Research,"May 22–25, 2023",48%,46%,2%,0%,0%,2%,1%,1%,IVR,2%,May 22,May,25,2023
3,Mainstreet Research,"May 21–24, 2023",49%,45%,2%,0%,0%,2%,1%,1%,IVR,4%,May 21,May,24,2023
4,Janet Brown Opinion Research,"May 12–24, 2023",52%,44%,1%,1%,1%,1%,0%,0%,Telephone/Online,8%,May 12,May,24,2023


In [268]:
polls['totalwords'] = polls['End Date'].str.split().str.len()
polls['New End Date'] = np.where(polls['totalwords'] == 1.0, polls['Start Month']+ ' ' +polls['End Date']+', '+polls['Year'], polls['End Date']+', '+polls['Year'])
polls.head()

,Pollster,Dates,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Polling method,Lead,Start Date,Start Month,End Date,Year,totalwords,New End Date
0,EKOS,"May 19–26, 2023",50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,Online/IVR,3.1%,May 19,May,26,2023,1.0,"May 26, 2023"
1,Leger,"May 23–25, 2023",49%,46%,0%,0%,0%,0%,0%,0%,Online,3%,May 23,May,25,2023,1.0,"May 25, 2023"
2,Mainstreet Research,"May 22–25, 2023",48%,46%,2%,0%,0%,2%,1%,1%,IVR,2%,May 22,May,25,2023,1.0,"May 25, 2023"
3,Mainstreet Research,"May 21–24, 2023",49%,45%,2%,0%,0%,2%,1%,1%,IVR,4%,May 21,May,24,2023,1.0,"May 24, 2023"
4,Janet Brown Opinion Research,"May 12–24, 2023",52%,44%,1%,1%,1%,1%,0%,0%,Telephone/Online,8%,May 12,May,24,2023,1.0,"May 24, 2023"


In [269]:
polls['totalwords start'] = polls['Start Date'].str.split().str.len()
polls['New Start Date'] = np.where(polls['totalwords start'] == 2.0, polls['Start Date']+', '+polls['Year'], polls['Start Date'])

In [270]:
polls = polls.drop(columns=['Start Month','Start Date', 'End Date', 'Year', 'totalwords', 'totalwords start', 'Dates', 'Pollster', 'Polling method'])
polls = polls.rename(columns={'New End Date': 'End Date', 'New Start Date': 'Start Date'})
polls['End Date'] = polls['End Date'].fillna(polls['Start Date'])

polls.head()

,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Lead,End Date,Start Date
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,3.1%,"May 26, 2023","May 19, 2023"
1,49%,46%,0%,0%,0%,0%,0%,0%,3%,"May 25, 2023","May 23, 2023"
2,48%,46%,2%,0%,0%,2%,1%,1%,2%,"May 25, 2023","May 22, 2023"
3,49%,45%,2%,0%,0%,2%,1%,1%,4%,"May 24, 2023","May 21, 2023"
4,52%,44%,1%,1%,1%,1%,0%,0%,8%,"May 24, 2023","May 12, 2023"


In [271]:
polls.isnull().any()

UCP           False
NDP           False
Alberta       False
Liberal       False
IPA           False
Green         False
WIP           False
Others        False
Lead          False
End Date      False
Start Date    False
dtype: bool

In [284]:
polls=polls[['UCP', 'NDP', 'Alberta', 'Liberal', 'IPA', 'Green', 'WIP', 'Others', 'Lead', 'Start Date', 'End Date']]
polls.head()

,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Lead,Start Date,End Date
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,3.1%,"May 19, 2023","May 26, 2023"
1,49%,46%,0%,0%,0%,0%,0%,0%,3%,"May 23, 2023","May 25, 2023"
2,48%,46%,2%,0%,0%,2%,1%,1%,2%,"May 22, 2023","May 25, 2023"
3,49%,45%,2%,0%,0%,2%,1%,1%,4%,"May 21, 2023","May 24, 2023"
4,52%,44%,1%,1%,1%,1%,0%,0%,8%,"May 12, 2023","May 24, 2023"


In [314]:
polls['Start Date'] = pd.to_datetime(polls['Start Date'], format=None)
polls['End Date'] = pd.to_datetime(polls['End Date'], format=None)

In [325]:
duplicated_rows = []

# Loop through each row in the DataFrame
for _, row in polls.iterrows():
    start_date = row['Start Date']
    end_date = row['End Date']

    # Loop through each day between the start and end dates (inclusive)
    for date in pd.date_range(start=start_date, end=end_date):
        duplicated_row = row.copy()  # Create a copy of the original row
        duplicated_row['Date'] = date  # Add a new 'Date' column with the current date
        duplicated_rows.append(duplicated_row)

# Create a new DataFrame from the duplicated rows
polls_new = pd.DataFrame(duplicated_rows)

In [327]:
polls_new = polls_new.drop(columns=(['Start Date', 'End Date', 'Lead']))
polls_new.head()

,UCP,NDP,Alberta,Liberal,IPA,Green,WIP,Others,Date
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,2023-05-19
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,2023-05-20
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,2023-05-21
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,2023-05-22
0,50.4%,47.3%,0.8%,0.4%,0%,0%,0%,1.1%,2023-05-23


In [366]:
polls_long = pd.melt(polls_new, id_vars=['Date'], var_name='Party', value_name='Support')
polls_long =polls_long.replace('–', 0)
polls_long['Support'] = polls_long['Support'].str.rstrip('%')
polls_long = polls_long.fillna(0)
polls_long['Party'] = polls_long['Party'].astype(str)
polls_long =polls_long.replace('3%[g]', 3)
polls_long['Support'] = polls_long['Support'].astype(float)

polls_long.head()

,Date,Party,Support
0,2023-05-19,UCP,50.4
1,2023-05-20,UCP,50.4
2,2023-05-21,UCP,50.4
3,2023-05-22,UCP,50.4
4,2023-05-23,UCP,50.4


In [376]:
polls_avg = polls_long.groupby(['Date', 'Party']).mean().reset_index()

# Visualizations

In [383]:
color_map = {'Alberta': 'yellow', 'Green': 'green', 'IPA': 'pink', 'Liberal': 'red', 'NDP': 'orange', 'UCP': 'blue', 'WIP': 'purple', 'Others': 'gray'}

# Plot the line chart with colors based on the 'Group' column
fig = px.line(polls_avg, x='Date', y='Support', color='Party', color_discrete_map=color_map)

fig.show()